# Get all crypto currencies from Binance live
## and save in a sql database

In [15]:
# !pip install python-binance -U


In [16]:
# import libraries
from IPython.display import display
import pandas as pd
import numpy as np
from binance.client import Client
import asyncio
from binance import AsyncClient, BinanceSocketManager
import nest_asyncio
nest_asyncio.apply()

In [17]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///CryptoDB.db')
client = Client()
info = client.get_exchange_info()
symbols = [s['symbol'] for s in info['symbols']]
exclude = ['UP', 'DOWN', 'BEAR', 'BULL']    
non_lev = [s for s in symbols if all(excludes not in s for excludes in exclude)]
# ends with 'USDT'
relevant = [s for s in non_lev if s.endswith('USDT')]

In [18]:
multi = [i.lower() + '@trade' for i in relevant]

In [19]:
def createframe(msg):
    df = pd.DataFrame([msg['data']])
    df['symbol'] = msg['stream']
    df = df.loc[:,['s','E','p']]
    df.columns=['symbol', 'Time', 'Price']
    df.Price = df.Price.astype(float)
    df.Time = pd.to_datetime(df.Time, unit='ms')
    return df

In [20]:
async def main():
    client = await AsyncClient.create()
    bm = BinanceSocketManager(client)
    ms = bm.multiplex_socket(multi)
    async with ms as tscm:
        while True:
            res = await tscm.recv()
            if res:
                frame = createframe(res)
                frame.to_sql(frame.symbol[0], engine, if_exists='append', index=False)
    await client.close_connection()
                

In [21]:
if __name__ == '__main__':
    loop = asyncio.get_event_loop()
    loop.run_until_complete(main())

KeyboardInterrupt: 